In [ ]:
!pip install transformers datasets accelerate peft bitsandbytes torch

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType

In [ ]:
# Nome do modelo
MODEL_NAME = "tiiuae/falcon-7b"

In [ ]:
# Carregar o tokenizador
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:

# Carregar o modelo com quantização para economizar memória
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,  # Usa menos VRAM
    device_map="auto"  # Distribui automaticamente entre CPU e GPU
)

In [ ]:
data = [
    {"input": "O que é inteligência artificial?", "output": "É a capacidade das máquinas de realizar tarefas que normalmente exigiriam inteligência humana."},
    {"input": "Quem foi Alan Turing?", "output": "Alan Turing foi um matemático e cientista da computação britânico, considerado um dos pais da computação moderna."}
]

In [ ]:
# Converter para Dataset Hugging Face
dataset = Dataset.from_list(data)

In [ ]:
def tokenize_function(examples):
    prompt = "Pergunta: " + examples["input"] + "\nResposta: " + examples["output"]

    # Tokenizar entrada e saída juntas
    tokens = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

    # Criar labels: rótulos são os mesmos input_ids, mas ignoramos o padding (-100)
    tokens["labels"] = tokens["input_ids"].copy()
    tokens["labels"] = [
        -100 if token == tokenizer.pad_token_id else token for token in tokens["labels"]
    ]

    return tokens


In [ ]:
# Tokenizar o dataset
dataset = dataset.map(tokenize_function)

In [ ]:
dataset

In [ ]:
# Configurar LoRA
lora_config = LoraConfig(
    r=8,  # Define o tamanho das matrizes auxiliares LoRA
    lora_alpha=32,  # Define a escala do ajuste LoRA
    lora_dropout=0.05,  # Adiciona dropout para evitar overfitting
    bias="none",
    task_type=TaskType.CAUSAL_LM  # Define o modelo como um "causal language model"
)

In [ ]:
# Aplicar LoRA ao Falcon 7B
model = get_peft_model(model, lora_config)

In [ ]:
# Exibir os parâmetros treináveis
model.print_trainable_parameters()

In [ ]:
# Configurar os hiperparâmetros do treinamento
training_args = TrainingArguments(
    output_dir="./falcon-7b-lora-finetuned",  # Onde salvar o modelo treinado
    per_device_train_batch_size=2,  # Usa batch pequeno para economizar VRAM
    gradient_accumulation_steps=4,  # Simula batch maior sem estourar a VRAM
    num_train_epochs=3,  # Número de épocas de treinamento
    learning_rate=2e-5,  # Taxa de aprendizado otimizada para LoRA (0.00002)
    logging_dir="./logs",  # Diretório de logs para análise
    logging_steps=10,  # Salvar logs a cada 10 steps
    save_strategy="epoch",  # Salvar checkpoints no final de cada época
    fp16=True,  # Usa FP16 para reduzir o consumo de VRAM
    push_to_hub=False,  # Se quiser salvar no Hugging Face, mude para True
    report_to="none"  # 🚀 Isso desativa o W&B corretamente!
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

# Iniciar o treinamento
trainer.train()

In [ ]:
# Salvar modelo treinado
model.save_pretrained("./falcon-7b-lora-finetuned")
tokenizer.save_pretrained("./falcon-7b-lora-finetuned")

In [ ]:
# Testar geração de texto
input_text = "Quem descobriu o Brasil?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

In [ ]:
# Gerar resposta com o modelo treinado
output = model.generate(
    input_ids,
    attention_mask=input_ids.ne(tokenizer.pad_token_id),
    max_length=500,
    temperature=1.0,
    top_p=0.9,
    repetition_penalty=1.2,  # 🚀 Penaliza palavras repetidas
    do_sample=True
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("\n🔹 Resposta Gerada:\n", generated_text)